# Optic Diseases
https://www.kaggle.com/datasets/gunavenkatdoddi/eye-diseases-classification/data

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Reading Dataset